In [2]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'

# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd

# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [39]:
### 법정동이 385개 정보밖에 없는데?
### 10프로 정도 뿐이라 분석에서 제외

In [6]:
file_list = os.listdir(f'{rawdata_path}/공동주택가격정보')
file_list

['AL_11_D167_20210716.csv',
 'AL_11_D167_20220428.csv',
 'AL_11_D167_20230130.csv',
 'AL_D167_11_20240125.csv',
 '2022.csv']

In [8]:
df_2022 = pd.read_csv(f'{rawdata_path}/공동주택가격정보/2022.csv')
df_2022.head()


,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공동주택코드,공동주택구분코드,공동주택구분명,특수지명,공동주택명,동명,층명,호명,전용면적,공시가격,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2022,1,3,3,빌라,NaN,청운벽산빌리지,10,1,1011,83.61,597000000,2022-07-26
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2022,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1022,83.61,630000000,2022-07-26
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2022,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1023,83.61,630000000,2022-07-26
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2022,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1024,83.61,630000000,2022-07-26
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2022,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1025,83.61,630000000,2022-07-26


In [10]:
df_2022['공동주택구분명'].unique()

array(['빌라', '다세대', '아파트'], dtype=object)

In [16]:
df_2022['기준월'].unique()

array([1])

In [13]:
df_2022 = df_2022[['법정동코드', '법정동명', '기준연도', '기준월', '공동주택구분명', '전용면적', '공시가격' ]]
df_2022.head()

,법정동코드,법정동명,기준연도,기준월,공동주택구분명,전용면적,공시가격
0,1111010100,서울특별시 종로구 청운동,2022,1,빌라,83.61,597000000
1,1111010100,서울특별시 종로구 청운동,2022,1,빌라,83.61,630000000
2,1111010100,서울특별시 종로구 청운동,2022,1,빌라,83.61,630000000
3,1111010100,서울특별시 종로구 청운동,2022,1,빌라,83.61,630000000
4,1111010100,서울특별시 종로구 청운동,2022,1,빌라,83.61,630000000


In [20]:
df_2022['법정동명'] = df_2022['법정동명'].apply(lambda x: x.replace('서울특별시 ', ''))
df_2022.head()

,법정동코드,법정동명,기준연도,기준월,공동주택구분명,전용면적,공시가격
0,1111010100,종로구 청운동,2022,1,빌라,83.61,597000000
1,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000
2,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000
3,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000
4,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000


In [33]:
df_2022['면적당가격'] = round(df_2022['공시가격'] / df_2022['전용면적'])
df_2022.head()

,법정동코드,법정동명,기준연도,기준월,공동주택구분명,전용면적,공시가격,면적당가격
0,1111010100,종로구 청운동,2022,1,빌라,83.61,597000000,7140294.0
1,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000,7534984.0
2,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000,7534984.0
3,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000,7534984.0
4,1111010100,종로구 청운동,2022,1,빌라,83.61,630000000,7534984.0


In [34]:
df_2022.groupby(['법정동명'])[['전용면적', '공시가격', '면적당가격']].mean()

,전용면적,공시가격,면적당가격
법정동명,,,
강남구 개포동,71.421052,1.383108e+09,1.887528e+07
강남구 논현동,63.473067,6.223789e+08,9.310565e+06
강남구 대치동,88.179157,1.779792e+09,1.861518e+07
강남구 도곡동,105.975951,1.820359e+09,1.734072e+07
강남구 삼성동,85.440386,1.390569e+09,1.551696e+07
...,...,...,...
중랑구 면목동,56.993715,3.225891e+08,5.451250e+06
중랑구 묵동,64.819527,3.627737e+08,5.452409e+06
중랑구 상봉동,56.278971,3.301450e+08,5.785088e+06


In [35]:
df_2022['법정동명'].nunique()

385

In [37]:
df_2022 = pd.read_csv(f'{rawdata_path}/공동주택가격정보/AL_D167_11_20240125.csv', encoding = 'cp949')
df_2022.head()

,고유번호,법정동코드,법정동명,특수지구분코드,특수지구분명,지번,기준연도,기준월,공동주택코드,공동주택구분코드,공동주택구분명,특수지명,공동주택명,동명,층명,호명,전용면적,공시가격,데이터기준일자
0,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2023,1,3,3,빌라,NaN,청운벽산빌리지,2,1,212,187.49,883000000,2024-01-20
1,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2023,1,3,3,빌라,NaN,청운벽산빌리지,10,3,1036,83.61,550000000,2024-01-20
2,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2023,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1027,83.61,567000000,2024-01-20
3,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2023,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1026,83.61,567000000,2024-01-20
4,1111010100100010000,1111010100,서울특별시 종로구 청운동,1,일반,1,2023,1,3,3,빌라,NaN,청운벽산빌리지,10,2,1023,83.61,567000000,2024-01-20


In [38]:
df_2022['법정동명'].nunique()

386